- # 从get_result文件导入已经写好的读取kv值的方法
- # 导入要解析的短信文本数据

In [1]:
import pandas as pd
from get_result import *
sms_data = pd.read_csv('data/after_filted.csv')

# 先调用之前的正则表达式文件先进行匹配，如果能够匹配到短句的信息就直接跳过，增加i,处理下一个句子

In [216]:
i = 182 
raw_sentence = sms_data.loc[i]['原句']
print('-'*50+"%d"%(i)+':类别'+'-'*50+'\n%s\n'%(sms_data.loc[i]['sence']))
print('-'*50+'短句'+'-'*50+'\n%s\n\n'%(sms_data.loc[i]['短句'])+'-'*100+'\n长句：%s\n'%(raw_sentence))
result = get_result(raw_sentence, kre_path='data/kre_914.csv')
print('='*100)
for k,v in result.items():
    print('{} : {}'.format(k, v))

--------------------------------------------------182:类别--------------------------------------------------
10052充值成功_score

--------------------------------------------------短句--------------------------------------------------
代理商于年月日时分为您充值.元，办理成功

----------------------------------------------------------------------------------------------------
长句：代理商13429747224于2018年07月15日12时00分为您充值100.00元，办理成功。您目前的可用实时余额为79.74交易流水号:1318071550736893。回复CXYE查询您当前话费余额，发10086到10086实时查询、办理各类业务，短信收发免费。感谢您选择中国移动。

话费余额 : 79.74
截止时间 : 2018年07月15日12时00分
缴费金额 : 100.00元


## 对将要写入的正则表达式的cn_key进行一个判断，以防不在规定的cn_key集合内
## 显示当前文件中cn_key对应的key的命名，以免key命名冲突

In [217]:
cn_key_set = ['可兑换积分','截止时间','缴费金额','缴费时间','话费余额','缴费渠道','充值号码',\
              '被充值号码','订单号','流水号','消费金额','账单月份','账户余额','备注']
cn_key  = '缴费金额'
print(cn_key in cn_key_set)
kre = pd.read_csv('data/kre_914.csv')
for j in kre.index:
    if cn_key == kre.loc[j]['cn_key']:
        print(kre.loc[j]['key'])

True
ldd_payment


In [219]:
import re
key = 'sqd_payment'
re_express = '充值(?P<{}>[\d\.]+[元块￥])'.format(key)
find =re.search(re_express, raw_sentence)
if find:
    print(find)
    print('----------------------------------------------\n\
        {} : {}\n -----------------------------------------------'.format(key, find.group(key)))
    print('key:{}\ncn_key:{}\nre:{}'.format(key, cn_key, re_express))
else:
    print('not find')


<_sre.SRE_Match object; span=(34, 43), match='充值100.00元'>
----------------------------------------------
        sqd_payment : 100.00元
 -----------------------------------------------
key:sqd_payment
cn_key:缴费金额
re:充值(?P<sqd_payment>[\d\.]+[元块￥])


# 把内容保存到kre数据中，其中last_commit是该句正则表达式所影响的句子，为以后改动正则表达式测试用

In [220]:
kre.loc[kre.index[-1]+1] =  {'key':key,
        'cn_key':cn_key,
        're':re_express,
        'last_commit':'原文ID:{}'.format(sms_data.loc[i]['原文ID']),
        'level':0}

In [221]:
kre

,cn_key,key,last_commit,re,level
0,截止时间,sqd_deadtime,[49],"截[至止].*?(?P<sqd_deadtime>(\d{4}年)?(\d{1,2}月)?(...",0
1,话费余额,sqd_call_balance,NaN,话费余额[：为是]?(?P<sqd_call_balance>.*?[元块￥]),0
2,话费余额,sqd_family_call_balance,NaN,家庭帐户.*余额[:为是](?P<sqd_family_call_balance>-?\d+...,0
3,消费金额,sqd_family_call_cost,NaN,家庭帐户.*消费[:为是]?(?P<sqd_family_call_cost>-?\d+\....,0
4,话费余额,sqd_call_balance2,[30],总余额.*?(?P<sqd_call_balance2>\d+.*?[元块￥]),0
5,话费余额,sqd_call_balance1,[40],可用余额[为是]?：?(?P<sqd_call_balance1>-?\d+.*?[元块￥]),0
6,账户余额,sqd_account_balance,[67],账户余额[：为是]?(?P<sqd_account_balance>\d+.*?[元块￥]),0
7,话费余额,ldd_charge_balance,NaN,(话费)?总?余额(已经)?只?还?剩?余?(为|是|了)?(:|：)?(?P<ldd_ch...,0
8,消费金额,ldd_charge_amount,NaN,(消费|产生话费)(合计)?(:|：)?\(?(?P<ldd_charge_amount>[...,0
9,缴费渠道,ldd_payment_channel,NaN,通过(?P<ldd_payment_channel>.*?)(及时)?[冲缴交充](值|纳)...,0


In [222]:
kre.to_csv('data/kre_914.csv', index=False)